## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [6]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), input_shape = (32, 32, 3), activation = 'relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3, 3), activation = 'relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100, activation = 'relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation= 'softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

/home/wulabasus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 12s 237us/step - loss: 1.3687 - accuracy: 0.5552
Epoch 2/100
50000/50000 [==============================] - 7s 149us/step - loss: 0.7821 - accuracy: 0.7280
Epoch 3/100
50000/50000 [==============================] - 8s 150us/step - loss: 0.5282 - accuracy: 0.8151
Epoch 4/100
50000/50000 [==============================] - 8s 153us/step - loss: 0.3302 - accuracy: 0.8868
Epoch 5/100
50000/50000 [==============================] - 8s 150us/step - loss: 0.2079 - accuracy: 0.9300
Epoch 6/100
50000/50000 [==============================] - 8s 150us/step - loss: 0.1524 - accuracy: 0.9477
Epoch 7/100
50000/50000 [==============================] - 8s 150us/step - loss: 0.1299 - accuracy: 0.9554
Epoch 8/100
50000/50000 [==============================] - 8s 150us/step - loss: 0.1211 - accuracy: 0.9586
Epoch 9/100
50000/50000 [==============================] - 8s 150us/step - loss: 0.1044 - accuracy: 0.9639
Epoch 10/100
50000/50000 [==========

50000/50000 [==============================] - 7s 150us/step - loss: 0.0135 - accuracy: 0.9958
Epoch 77/100
50000/50000 [==============================] - 8s 151us/step - loss: 0.0076 - accuracy: 0.9977
Epoch 78/100
50000/50000 [==============================] - 7s 150us/step - loss: 0.0152 - accuracy: 0.9957
Epoch 79/100
50000/50000 [==============================] - 8s 151us/step - loss: 0.0176 - accuracy: 0.9951
Epoch 80/100
50000/50000 [==============================] - 8s 150us/step - loss: 0.0167 - accuracy: 0.9954
Epoch 81/100
50000/50000 [==============================] - 8s 150us/step - loss: 0.0142 - accuracy: 0.9960
Epoch 82/100
50000/50000 [==============================] - 8s 150us/step - loss: 0.0178 - accuracy: 0.9949
Epoch 83/100
50000/50000 [==============================] - 8s 151us/step - loss: 0.0074 - accuracy: 0.9976
Epoch 84/100
50000/50000 [==============================] - 8s 150us/step - loss: 0.0091 - accuracy: 0.9972
Epoch 85/100
50000/50000 [===============

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[6.0455627e-09, 2.0930256e-15, 2.1434605e-12, 9.9998987e-01,
        1.0107487e-05, 8.9032258e-16, 1.2281121e-11, 6.8939445e-13,
        1.7985649e-09, 1.1705200e-17]], dtype=float32)